# Connect Colab to a Chameleon server

This notebook describes how to connect Colab to a server running on Chameleon. This allows you to run experiments requiring bare metal access, storage, memory, GPU and compute that exceeds the abilities of Colab's hosted runtime, but with Colab's familiar interface (and notebooks stored in your Google Drive). It also allows you to easily go back and forth between the convenience of Colab's hosted runtime and Chameleon's greater capabilities, depending on the needs of your experiment.

## Provision the resource


### Check resource availability

This notebook will try to reserve a bare metal Ubuntu server with RTX6000 GPU on CHI@UC - pending availability. Before you begin, you should check the host calendar at [https://chi.uc.chameleoncloud.org/project/leases/calendar/host/](https://chi.uc.chameleoncloud.org/project/leases/calendar/host/). In the "Node Type" dropdown, filter on `gpu_rtx_6000` and make sure some hosts are available.

### Chameleon configuration

You can change your Chameleon project name (if not using the one that is automatically configured in the JupyterHub environment) and the site on which to reserve resources (depending on availability) in the following cell.

In [ ]:
import chi, os, time, datetime
from chi import lease
from chi import server
from chi import context
from chi import hardware

context.version = "1.0" # required during transition
context.choose_site(default="CHI@UC")
context.choose_project()
username = os.getenv('USER') # all exp resources will have this prefix

If you need to change the details of the Chameleon server, e.g. use a different GPU type depending on availability, you can do that in the following cell.

In [ ]:
node_type = "gpu_rtx_6000"

### Reservation

The following cells will create a reservation that begins now, and ends in 8 hours, *if* your requested node type is available.

If the node type you have requested is *not* available right now, it will start your reservation as soon as one is available.

You can refer to the [CHI@UC host calendar](https://chi.uc.chameleoncloud.org/project/leases/calendar/host/) to see availability (change the "Node type" selection to `gpu_rtx_6000`), or refer to the [CHI@TACC host calendar](https://chi.tacc.chameleoncloud.org/project/leases/calendar/host/) to see availability for other node types with GPU.

In [ ]:
gpu_start_times = [n.next_free_timeslot()[0] for n in hardware.get_nodes(node_type=node_type)]
current_time = datetime.datetime.now(datetime.timezone.utc) + datetime.timedelta(minutes=1)
if min(gpu_start_times) > current_time:
    lease_start = min(gpu_start_times)
    print(f"There is no {node_type} available now, you can request one starting at {str(lease_start)} (UTC).")
else:
    lease_start = current_time
    print(f"A {node_type} IS available now, your lease will start right away.")


In [ ]:
l = lease.Lease(f"colab-{username}-{node_type}", duration=datetime.timedelta(hours=8),
                start_date = lease_start  )
l.add_node_reservation(node_type = node_type, amount = 1) 
l.add_fip_reservation(1) 
l.submit(idempotent=True)

### Provisioning resources

This section provisions resources. It will take approximately 10 minutes. You can check on its status in the Chameleon web-based UI: [https://chi.uc.chameleoncloud.org/project/instances/](https://chi.uc.chameleoncloud.org/project/instances/), then come back here when it is in the READY state.

In [ ]:
# continue here, whether using a lease created just now or one created earlier
l = lease.get_lease(f"colab-{username}-{node_type}")

In [ ]:
s = server.Server(
    f"colab-{username}-{node_type}", 
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu22.04"
)
s.submit(idempotent=True)

Associate an IP address with this server:

In [ ]:
reserved_fip = l.get_reserved_floating_ips()[0]
s.associate_floating_ip(reserved_fip)

and wait for it to come up:

In [ ]:
s.check_connectivity(host=reserved_fip)

## Install stuff

The following cells will install some basic packages in order to connect your Colab frontend to your Chameleon server. However, you may want to log in to your Chameleon server in order to access its terminal and install or configure packages outside of Colab.

To log in to the resource, use File > New > Terminal in the Chameleon JupyterHub environment, or your local terminal, and run:


In [ ]:
print("ssh cc@" + reserved_fip)

Meanwhile, install an updated GPU driver on your resource:

In [ ]:
from chi import ssh

node = ssh.Remote(reserved_fip)

In [ ]:
node.run('wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.0-1_all.deb')
node.run('sudo dpkg -i cuda-keyring_1.0-1_all.deb')
node.run('sudo apt update')
node.run('sudo apt -y install linux-headers-$(uname -r)')
node.run('sudo apt -y install nvidia-driver-560') 
node.run('sudo apt -y install cuda-12-2 cuda-runtime-12-2 cuda-drivers=560.35.03-1')
node.run('sudo apt -y install nvidia-gds-12-2') 
node.run('sudo apt -y install libcudnn8=8.9.7.29-1+cuda12.2 cuda-toolkit-12-2') # make sure the get cuda-12-2 version

In [ ]:
node.run("echo 'PATH=\"/usr/local/cuda-12.2/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin\"' | sudo tee /etc/environment")

Now we have to reboot, and make sure we have the specified CUDA:

In [ ]:
try:
    node.run('sudo reboot') # reboot and wait for it to come up
except:
    pass

In [ ]:
s.check_connectivity(host=reserved_fip)

In [ ]:
node = ssh.Remote(reserved_fip) # note: need a new SSH session to get new PATH
node.run('nvidia-smi')
node.run('nvcc --version')

## Set up Docker

To use common deep learning frameworks like Tensorflow or PyTorch, we can run *containers* that have all the prerequisite libraries necessary for these frameworks. Here, we will set up the container framework.

In [ ]:
node.run("curl -sSL https://get.docker.com/ | sudo sh")
node.run("sudo groupadd -f docker; sudo usermod -aG docker $USER")

In [ ]:
node = ssh.Remote(reserved_fip) # note: need a new SSH session to get new group membership
node.run("docker run hello-world")

In [ ]:
# get NVIDIA container toolkit 
node.run("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list")
node.run("sudo apt update")
node.run("sudo apt-get install -y nvidia-container-toolkit")
node.run("nvidia-ctk runtime configure --runtime=docker")
node.run("sudo systemctl restart docker")

In [ ]:
# check that we can see GPU from inside container
node.run("docker run --rm --gpus all ubuntu nvidia-smi")

## Start a Jupyter server

Now, you can start a Jupyter server with your preferred deep learning framework.

For Tensorflow, run:

In [ ]:
node.run("docker run -d --rm -p 8888:8888 --gpus all quay.io/jupyter/tensorflow-notebook:cuda-tensorflow-2.16.1")

Then, after a minute or two, check the logs of this process:

In [ ]:
node.run("docker ps -q | xargs -L 1 docker logs")

In the output of this command, look for a line like

```
        http://127.0.0.1:8888/?token=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
```

You will copy and paste this URL into your own browser, but replace the **127.0.0.1** with the floating IP assigned to your server. This will open a Jupyter instance running *on your GPU server*.

If you start a notebook inside this Jupyter server, you should be able to run

```
import tensorflow as tf
print(tf.config.list_physical_devices("GPU"))
```

and see a GPU device listed.

To stop the running container(s), use:

In [ ]:
node.run("docker ps -q | xargs -L 1 docker container stop")

For PyTorch, run:

In [ ]:

node.run("docker run -d -p 8888:8888 --rm --gpus all quay.io/jupyter/pytorch-notebook:cuda12-latest")

Then, after a minute or two, check the logs of this process:

In [ ]:
node.run("docker ps -q | xargs -L 1 docker logs")

In the output of this command, look for a line like

```
        http://127.0.0.1:8888/?token=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
```

You will copy and paste this URL into your own browser, but replace the **127.0.0.1** with the floating IP assigned to your server. This will open a Jupyter instance running *on your GPU server*.

If you start a notebook inside this Jupyter server, you should be able to run

```
import torch
print(torch.cuda.get_device_name(0))
```

and see the name of your GPU.

To stop the running container(s), use:

In [ ]:
node.run("docker ps -q | xargs -L 1 docker container stop")

## Connect a Colab frontend to your Jupyter server

In [ ]:
print('ssh -L 127.0.0.1:8888:127.0.0.1:8888 cc@' + reserved_fip) 

In a **local terminal on your own laptop**, run the SSH command that is printed by the previous cell. This will set up a tunnel to the Jupyter server that you are running on a Chameleon instance.

If your Chameleon key is not in the default location, you should also specify the path to your key as an argument, using `-i`. For example:

In [ ]:
print('ssh -i ~/.ssh/id_rsa_chameleon -L 127.0.0.1:8888:127.0.0.1:8888 cc@' + reserved_fip) 

Leave this SSH session open.

Then, start your preferred container (un-comment one option)

In [ ]:
#node.run("docker run -d --rm -p 8888:8888 --gpus all quay.io/jupyter/tensorflow-notebook:cuda-tensorflow-2.16.1")
#node.run("docker run -d -p 8888:8888 --rm --gpus all quay.io/jupyter/pytorch-notebook:cuda12-latest")

and check the logs -

In [ ]:
node.run("docker ps -q | xargs -L 1 docker logs")

Look for a URL in this format:
    
```
http://127.0.0.1:8888/?token=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
```

Copy this URL - you will need it in the next step.

Now, you can open Colab in a browser. Click on the drop-down menu for "Connect" in the top right and select "Connect to a local runtime". Paste the URL you copied earlier into the space and click "Connect". Your notebook should now be running on your Colab host (you can put `!hostname` in a cell and run it to verify!)


## Release resources

If you finish with your experimentation before your lease expires,release your resources and tear down your environment by running the following (commented out to prevent accidental deletions).

This section is designed to work as a "standalone" portion - you can come back to this notebook, ignore the top part, and just run this section to delete your reasources.

In [ ]:
# setup environment - if you made any changes in the top part, make the same changes here
import chi, os
from chi import lease, magic, context

context.version = "1.0" # required during transition

context.choose_site(default="CHI@UC")
context.choose_project()
username = os.getenv('USER') # all exp resources will have this prefix

node_type = "gpu_rtx_6000"
lease = lease.get_lease(f"colab-{username}-{node_type}")

In [ ]:
# un-comment to free resources
# chi.magic.cleanup_resources(lease.id)
